# Summer of Code - Artificial Intelligence

## Week 10: Deep Learning

### Day 05: Machine Translation

In this notebook, we will explore **Machine Translation** using **Bidirectional RNNs** in PyTorch.


# Neural Machine Translation

Neural Machine Translation (NMT) is another NLP task where we translate text from one language to another using neural networks.


## Encoder-Decoder Architecture

The most common architecture for NMT is the Encoder-Decoder architecture. The encoder processes the input sentence and encodes it into a fixed-length context vector, which is then used by the decoder to generate the translated sentence.

<img src="images/enc_dec.png" alt="Encoder-Decoder Architecture" width="600"/>


### English to Urdu Translation Dataset


In [ ]:
from zipfile import ZipFile
from urllib.request import urlopen, Request


url = "https://www.manythings.org/anki/urd-eng.zip"
req = Request(
    url,
    headers={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    },
)
with urlopen(req) as response:
    with open("urd-eng.zip", "wb") as f:
        f.write(response.read())
with ZipFile("urd-eng.zip", "r") as zip_ref:
    zip_ref.extractall("./eng-urd")
print("Dataset downloaded and extracted.")

In [3]:
with open("./eng-urd/urd.txt", "r", encoding="utf-8") as f:
    data = f.readlines()

data[:5]

['Hi.\tسلام۔\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #9020897 (nusrat)\n',
 'Help!\tمدد۔\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #1462368 (nabeel_tahir)\n',
 'Thanks.\tشکریہ۔\tCC-BY 2.0 (France) Attribution: tatoeba.org #2057650 (nava) & #9020893 (nusrat)\n',
 'We won.\tہم جیت گئے۔\tCC-BY 2.0 (France) Attribution: tatoeba.org #2107675 (CK) & #2123755 (nabeel_tahir)\n',
 'Beat it.\tبھاگ جائو۔\tCC-BY 2.0 (France) Attribution: tatoeba.org #37902 (CM) & #1610833 (nabeel_tahir)\n']

In [4]:
source_target = [line.strip().split("\t")[:2] for line in data]
source_target[:3]

[['Hi.', 'سلام۔'], ['Help!', 'مدد۔'], ['Thanks.', 'شکریہ۔']]

In [28]:
import numpy as np


np.random.shuffle(source_target)
source_target[:3]

[['Hey, look at this.', 'یہ دیکھو۔'],
 ['I keep a dog.', 'میں کتا رکھتا ہوں۔'],
 ["I just don't know what to say.", 'مجھے ابھی نہیں پتا کیا کہنا ہے ۔']]

In [31]:
source_sentences, target_sentences = zip(*source_target)
for i in range(3):
    print(f"{source_sentences[i]} => {target_sentences[i]}")

Hey, look at this. => یہ دیکھو۔
I keep a dog. => میں کتا رکھتا ہوں۔
I just don't know what to say. => مجھے ابھی نہیں پتا کیا کہنا ہے ۔


### Build Vocabulary


In [33]:
def tokenize(sentences):
    return [s.lower().split() for s in sentences]

In [37]:
source_tokens = tokenize(source_sentences)
target_tokens = tokenize(target_sentences)

source_tokens[:3], target_tokens[:3]

([['hey,', 'look', 'at', 'this.'],
  ['i', 'keep', 'a', 'dog.'],
  ['i', 'just', "don't", 'know', 'what', 'to', 'say.']],
 [['یہ', 'دیکھو۔'],
  ['میں', 'کتا', 'رکھتا', 'ہوں۔'],
  ['مجھے', 'ابھی', 'نہیں', 'پتا', 'کیا', 'کہنا', 'ہے', '۔']])

In [38]:
from collections import Counter


def build_vocabulary(tokenized_sentences, source=True, max_vocab_size=1000):
    word_counts = Counter()
    for s in tokenized_sentences:
        word_counts.update(s)

    # Create vocabulary with special tokens
    if source:
        vocab = {"<PAD>": 0, "<UNK>": 1}
        most_common = word_counts.most_common(max_vocab_size - 2)
    else:
        vocab = {"<PAD>": 0, "<UNK>": 1, "<SOS>": 2, "<EOS>": 3}
        most_common = word_counts.most_common(max_vocab_size - 4)
    # Add most common words
    for word, _ in most_common:
        vocab[word] = len(vocab)

    return vocab, word_counts


source_vocab, source_word_counts = build_vocabulary(source_tokens, max_vocab_size=1500)
target_vocab, target_word_counts = build_vocabulary(
    target_tokens, source=False, max_vocab_size=1500
)
print("English Vocabulary Size:", len(source_vocab))
print("Urdu Vocabulary Size:", len(target_vocab))
print("Most common English words:", source_word_counts.most_common(5))
print("Most common Urdu words:", target_word_counts.most_common(5))

English Vocabulary Size: 1500
Urdu Vocabulary Size: 1500
Most common English words: [('i', 284), ('the', 265), ('to', 223), ('you', 195), ('a', 167)]
Most common Urdu words: [('میں', 376), ('ہے۔', 324), ('نے', 182), ('اس', 155), ('وہ', 149)]


In [39]:
print(target_vocab)

{'<PAD>': 0, '<UNK>': 1, '<SOS>': 2, '<EOS>': 3, 'میں': 4, 'ہے۔': 5, 'نے': 6, 'اس': 7, 'وہ': 8, 'کے': 9, 'کو': 10, 'نہیں': 11, 'کی': 12, 'سے': 13, 'ٹام': 14, 'مجھے': 15, 'تم': 16, 'کیا': 17, 'کہ': 18, 'ہوں۔': 19, 'کا': 20, 'ہو': 21, 'ہے': 22, 'یہ': 23, 'آپ': 24, 'ایک': 25, 'کر': 26, 'ہے؟': 27, 'تھا۔': 28, 'میرے': 29, 'اپنی': 30, 'ہیں۔': 31, 'بہت': 32, 'اور': 33, 'رہا': 34, 'گا۔': 35, 'کافی': 36, 'پہ': 37, 'میری': 38, 'کچھ': 39, 'اسے': 40, 'بھی': 41, 'ہم': 42, 'گھر': 43, 'ہو؟': 44, 'گیا': 45, 'مریم': 46, 'تمہیں': 47, 'نہ': 48, 'ہوا': 49, 'تھی۔': 50, 'زیادہ': 51, 'ہو۔': 52, 'کرنا': 53, 'رہی': 54, 'رہے': 55, 'ہی': 56, 'اپنے': 57, 'کرنے': 58, 'سکتے': 59, 'میرا': 60, 'سال': 61, 'وقت': 62, 'گیا۔': 63, 'گی۔': 64, 'پسند': 65, 'پاس': 66, 'تک': 67, 'تھا': 68, 'کوئی': 69, 'جا': 70, 'لئیے': 71, 'ابھی': 72, 'آج': 73, 'کبھی': 74, 'تو': 75, 'پر': 76, 'یہاں': 77, 'ساتھ': 78, '۔': 79, 'کسی': 80, 'جلدی': 81, 'تمھیں': 82, 'آ': 83, 'سب': 84, 'گاڑی': 85, 'گئے': 86, 'کرتا': 87, 'جائو': 88, 'کام': 89, 'جب': 

In [40]:
train_size = int(0.9 * len(source_tokens))

X_train_tokens = list(source_tokens[:train_size])
X_val_tokens = list(source_tokens[train_size:])

X_train_dec_tokens = [
    ["<SOS>"] + sentence.copy()
    for sentence in target_tokens[:train_size]
]
X_val_dec_tokens = [
    ["<SOS>"] + sentence.copy()
    for sentence in target_tokens[train_size:]
]

y_train_tokens = [
    sentence.copy() + ["<EOS>"]
    for sentence in target_tokens[:train_size]
]
y_val_tokens = [
    sentence.copy() + ["<EOS>"]
    for sentence in target_tokens[train_size:]
]

print("X_train sample:", X_train_tokens[0])
print("X_train_dec sample:", X_train_dec_tokens[0])
print("y_train sample:", y_train_tokens[0])

X_train sample: ['hey,', 'look', 'at', 'this.']
X_train_dec sample: ['<SOS>', 'یہ', 'دیکھو۔']
y_train sample: ['یہ', 'دیکھو۔', '<EOS>']


In [41]:
def pad_tokens(sentence_tokens, max_length=15):
    padded_tokens = []
    for tokens in sentence_tokens:
        if len(tokens) > max_length:
            tokens = tokens[:max_length]
        else:
            tokens = tokens + ["<PAD>"] * (max_length - len(tokens))
        padded_tokens.append(tokens)
    return padded_tokens

In [42]:
pad_tokens(y_train_tokens[:2])

[['یہ',
  'دیکھو۔',
  '<EOS>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>'],
 ['میں',
  'کتا',
  'رکھتا',
  'ہوں۔',
  '<EOS>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>']]

In [44]:
def to_sequence(sentence_tokens, vocab, max_length=15):
    padded_tokens = pad_tokens(sentence_tokens, max_length)
    sequences = []
    for tokens in padded_tokens:
        sequence = [vocab.get(token, vocab["<UNK>"]) for token in tokens]
        sequences.append(sequence)
    return sequences


sample_text = ["I am happy", "This is a test sentence"]
sample_tokens = tokenize(sample_text)
sample_sequence = to_sequence(sample_tokens, source_vocab, max_length=10)
print("Sample text:", sample_sequence)

Sample text: [[2, 41, 1023, 0, 0, 0, 0, 0, 0, 0], [15, 8, 6, 1, 1, 0, 0, 0, 0, 0]]


In [45]:
X_train = to_sequence(X_train_tokens, source_vocab, max_length=15)
X_val = to_sequence(X_val_tokens, source_vocab, max_length=15)

X_train_dec = to_sequence(X_train_dec_tokens, target_vocab, max_length=15)
X_val_dec = to_sequence(X_val_dec_tokens, target_vocab, max_length=15)

y_train = to_sequence(y_train_tokens, target_vocab, max_length=15)
y_val = to_sequence(y_val_tokens, target_vocab, max_length=15)


print("X_train sample:", X_train[0])
print("y_train_dec sample:", X_train_dec[0])
print("y_train sample:", y_train[0])

X_train sample: [773, 59, 32, 191, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
y_train_dec sample: [2, 23, 298, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
y_train sample: [23, 298, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [47]:
import torch
from torch.utils.data import TensorDataset

train_data = TensorDataset(
    torch.tensor(X_train, dtype=torch.long),
    torch.tensor(X_train_dec, dtype=torch.long),
    torch.tensor(y_train, dtype=torch.long),
)

val_data = TensorDataset(
    torch.tensor(X_val, dtype=torch.long),
    torch.tensor(X_val_dec, dtype=torch.long),
    torch.tensor(y_val, dtype=torch.long),
)
print("Number of training samples:", len(train_data))
print("Number of validation samples:", len(val_data))

Number of training samples: 1034
Number of validation samples: 115


In [48]:
from torch.utils.data import DataLoader


batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

In [65]:
import torch.nn as nn


class Encoder(nn.Module):

    def __init__(
        self,
        input_vocab_size,
        embed_size,
        hidden_size,
        num_layers=2,
        bidirectional=False,
    ):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(
            input_vocab_size, embed_size, padding_idx=0
        )
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.gru = nn.GRU(
            embed_size,
            hidden_size,
            num_layers,
            batch_first=True,
            bidirectional=bidirectional,
            dropout=0.5 if num_layers > 1 else 0,
        )
        self.embed_dropout = nn.Dropout(0.5)
        if bidirectional:
            self.hidden_projection = nn.Linear(hidden_size * 2, hidden_size)

    def forward(self, x):
        embedded = self.embed_dropout(self.embedding(x))
        outputs, hidden = self.gru(embedded)
        if self.bidirectional:
            # Reshape: (num_layers * 2, batch, hidden) -> (num_layers, 2, batch, hidden)
            hidden = hidden.view(self.num_layers, 2, -1, self.hidden_size)
            # Concatenate forward and backward
            hidden = torch.cat([hidden[:, 0, :, :], hidden[:, 1, :, :]], dim=2)
            # Project to decoder size
            hidden = self.hidden_projection(hidden)
        return outputs, hidden


class Decoder(nn.Module):
    def __init__(self, output_vocab_size, embed_size, hidden_size, num_layers=2):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(
            output_vocab_size, embed_size, padding_idx=0
        )
        self.gru = nn.GRU(
            embed_size,
            hidden_size,
            num_layers,
            batch_first=True,
            dropout=0.5 if num_layers > 1 else 0,
        )
        self.embed_dropout = nn.Dropout(0.5)

    def forward(self, x, hidden):
        embedded = self.embed_dropout(self.embedding(x))
        outputs, hidden = self.gru(embedded, hidden)
        return outputs, hidden

In [50]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.fc = nn.Linear(decoder.hidden_size, len(target_vocab))
        self.dropout = nn.Dropout(0.6)

    def forward(self, source, target):
        _, encoder_hidden = self.encoder(source)
        decoder_outputs, _ = self.decoder(target, encoder_hidden)
        decoder_outputs = self.dropout(decoder_outputs)
        return self.fc(decoder_outputs)

# Training and Evaluation


In [51]:
import tqdm


def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    progress_bar = tqdm.tqdm(dataloader, desc="Training")
    for enc_inputs, dec_inputs, targets in progress_bar:
        # Move data to device
        enc_inputs, dec_inputs, targets = (
            enc_inputs.to(device),
            dec_inputs.to(device),
            targets.to(device),
        )

        outputs = model(enc_inputs, dec_inputs)
        outputs = outputs.reshape(-1, outputs.size(-1))
        targets = targets.reshape(-1)

        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        non_pad_mask = targets != 0  # Create mask for non-padding tokens
        total += non_pad_mask.sum().item()
        correct += ((predicted == targets) & non_pad_mask).sum().item()
        total_loss += loss.item()

        # Update progress bar
        progress_bar.set_postfix(
            {"loss": f"{loss.item():.4f}", "acc": f"{100 * correct / total:.2f}%"}
        )

    avg_loss = total_loss / len(dataloader)
    accuracy = 100 * correct / total

    return avg_loss, accuracy

## Step 11: Validation Function

The validation function evaluates the model without updating weights. This helps us monitor overfitting and select the best model.


In [52]:
def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    progress_bar = tqdm.tqdm(dataloader, desc="Validating")
    with torch.no_grad():

        for enc_inputs, dec_inputs, targets in progress_bar:
            # Move data to device
            enc_inputs, dec_inputs, targets = (
                enc_inputs.to(device),
                dec_inputs.to(device),
                targets.to(device),
            )

            outputs = model(enc_inputs, dec_inputs)
            outputs = outputs.reshape(-1, outputs.size(-1))
            targets = targets.reshape(-1)

            loss = criterion(outputs, targets)

            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            non_pad_mask = targets != 0  # Create mask for non-padding tokens
            total += non_pad_mask.sum().item()
            correct += ((predicted == targets) & non_pad_mask).sum().item()
            total_loss += loss.item()

            # Update progress bar with current accuracy
            progress_bar.set_postfix(
                {"loss": f"{loss.item():.4f}", "acc": f"{100 * correct / total:.2f}%"}
            )

    avg_loss = total_loss / len(dataloader)
    accuracy = 100 * correct / total

    return avg_loss, accuracy

In [70]:
# Instantiate the models
embed_size = 256
hidden_size = 512
num_layers = 2
encoder = Encoder(
    len(source_vocab), embed_size, hidden_size, num_layers, bidirectional=True
)
decoder = Decoder(len(target_vocab), embed_size, hidden_size, num_layers)

model = EncoderDecoder(encoder, decoder)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model

EncoderDecoder(
  (encoder): Encoder(
    (embedding): Embedding(1500, 256, padding_idx=0)
    (gru): GRU(256, 512, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
    (embed_dropout): Dropout(p=0.5, inplace=False)
    (hidden_projection): Linear(in_features=1024, out_features=512, bias=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(1500, 256, padding_idx=0)
    (gru): GRU(256, 512, num_layers=2, batch_first=True, dropout=0.5)
    (embed_dropout): Dropout(p=0.5, inplace=False)
  )
  (fc): Linear(in_features=512, out_features=1500, bias=True)
  (dropout): Dropout(p=0.6, inplace=False)
)

In [71]:
from torchinfo import summary

input_sample = torch.zeros((1, 15), dtype=torch.long).to(device)
summary(model, input_data=(input_sample, input_sample))

Layer (type:depth-idx)                   Output Shape              Param #
EncoderDecoder                           [1, 15, 1500]             --
├─Encoder: 1-1                           [1, 15, 1024]             --
│    └─Embedding: 2-1                    [1, 15, 256]              384,000
│    └─Dropout: 2-2                      [1, 15, 256]              --
│    └─GRU: 2-3                          [1, 15, 1024]             7,090,176
│    └─Linear: 2-4                       [2, 1, 512]               524,800
├─Decoder: 1-2                           [1, 15, 512]              --
│    └─Embedding: 2-5                    [1, 15, 256]              384,000
│    └─Dropout: 2-6                      [1, 15, 256]              --
│    └─GRU: 2-7                          [1, 15, 512]              2,758,656
├─Dropout: 1-3                           [1, 15, 512]              --
├─Linear: 1-4                            [1, 15, 1500]             769,500
Total params: 11,911,132
Trainable params: 11,911,1

In [72]:
import torch.optim as optim


# Loss function (ignore padding tokens)
criterion = nn.CrossEntropyLoss(ignore_index=0)

# Optimizer
learning_rate = 0.0007
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=5
)

print(f"Number of training batches: {len(train_loader)}")
print(f"Number of validation batches: {len(val_loader)}")

Number of training batches: 17
Number of validation batches: 2


In [73]:
num_epochs = 150
best_val_acc = 0.0
patience = 15
patience_counter = 0

# Track training history
train_losses = []
train_accs = []
val_losses = []
val_accs = []

print("Starting training...")
print(f"Device: {device}")
print(f"Number of epochs: {num_epochs}\n")

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    # Train
    train_loss, train_acc = train_epoch(
        model, train_loader, criterion, optimizer, device
    )
    # Validate
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    # Track history
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    scheduler.step(val_loss)

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        torch.save(model.state_dict(), "nmt_model.pth")
        print(f"  ✓ Saved best model (Val Acc: {val_acc:.2f}%)")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch + 1} epochs")
            break

print("Training complete!")
print(f"Best validation accuracy: {best_val_acc:.2f}%")

Starting training...
Device: cuda
Number of epochs: 150

Epoch 1/150


Validating: 100%|██████████| 2/2 [00:00<00:00, 46.48it/s, loss=5.2349, acc=14.41%]


  ✓ Saved best model (Val Acc: 14.41%)
Epoch 2/150


Validating: 100%|██████████| 2/2 [00:00<00:00, 45.91it/s, loss=5.0170, acc=14.62%]


  ✓ Saved best model (Val Acc: 14.62%)
Epoch 3/150


...

Epoch 54/150


Validating: 100%|██████████| 2/2 [00:00<00:00, 53.00it/s, loss=3.8667, acc=42.17%]


Epoch 55/150


Validating: 100%|██████████| 2/2 [00:00<00:00, 48.15it/s, loss=3.8690, acc=42.06%]

Early stopping triggered after 55 epochs
Training complete!
Best validation accuracy: 42.17%


# Load Best Model


In [74]:
model.load_state_dict(torch.load("nmt_model.pth", weights_only=True))
model = model.to(device)
validate(model, val_loader, criterion, device)

Validating: 100%|██████████| 2/2 [00:00<00:00, 12.77it/s, loss=3.8332, acc=42.17%]


(3.904155969619751, 42.16614090431125)

In [80]:
target_i2t = {idx: token for token, idx in target_vocab.items()}


def translate(sentence, model, source_vocab, target_vocab, max_length=20, device="cpu"):
    model.eval()
    sentence_tokens = sentence.lower().split()
    sequence = to_sequence([sentence_tokens], source_vocab, max_length=max_length)
    encoder_input = torch.tensor(sequence, dtype=torch.long).to(device)

    # Start with <SOS> token
    decoder_input = [target_vocab["<SOS>"]]
    translation = []

    with torch.no_grad():
        _, encoder_hidden = model.encoder(encoder_input)

        for _ in range(max_length):
            # Prepare decoder input
            dec_input = torch.tensor([decoder_input], dtype=torch.long).to(device)

            # Decode
            decoder_outputs, _ = model.decoder(dec_input, encoder_hidden)

            # Get prediction for the last token
            output = model.fc(decoder_outputs[:, -1, :])
            predicted_id = output.argmax(dim=-1).item()

            # Check for EOS token
            if predicted_id == target_vocab["<EOS>"]:
                break

            # Get the predicted word
            predicted_word = target_i2t.get(predicted_id, "")

            # Skip special tokens in output
            if predicted_word not in ["<PAD>", "<UNK>", "<SOS>", "<EOS>"]:
                translation.append(predicted_word)

            # Add predicted token to decoder input for next iteration
            decoder_input.append(predicted_id)
    return " ".join(translation)


# Test the translation function
test_sentences = source_sentences[:10]

print("Testing translations:\n")
for sentence in test_sentences:
    translated = translate(sentence, model, source_vocab, target_vocab, device=device)
    print(f"Source: {sentence}")
    print(f"Target: {translated}")
    print()

Testing translations:

Source: Hey, look at this.
Target: یہ دیکھو۔

Source: I keep a dog.
Target: میں کتا رکھتا ہوں۔

Source: I just don't know what to say.
Target: مجھے پتا ہے کہ تم امیر ہو۔

Source: How is it going?
Target: کيسا چل رہا ہے ؟

Source: Trust me!
Target: مجھ پر بھروسہ

Source: Tom's scared.
Target: ٹام مر ہے۔

Source: I'm seeing them tonight.
Target: مجھے نیند سے الرجی ہے۔

Source: What did she do today?
Target: انہیوں نے آج کیا کیا؟

Source: He decided to submit his resignation.
Target: اس نے اپنا استعفی کرانے کرانے

Source: His car is two years old.
Target: اس کی گاڑی دو پرانی ہے۔

